## Notebook E: Calculating Neighbourhood Reach Centrality
**TU Delft**<br>
**Author:** Ruth Nelson <br>

The purpose of this notebook is to calculate the Neighbourhood Reach Centrality for every neighbourhood.

1. Import libraries
2. Import dataframes
3. Create a NetworkX graph
4. Preparing network for Reach Centrality calculations
5. Calculate Reach Centrality
6. Save files

## 1. Import 

In [ ]:
import geopandas as gpd
import pandas as pd
import networkx as nx
import os
import glob

## 2. Import Library

Import the Spatial Justice Module with the Neighbourhood Reach function

In [ ]:
path_lib = " "

In [ ]:
os.chdir(path_lib)
for file in glob.glob("*"):
    print(file)

In [ ]:
import spatial_justice as sj

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', None)
#pd.set_option('display.notebook_repr_html', True)

### Import the dataframes

In [ ]:
path_edges = ""

In [ ]:
os.chdir(path_edges)
for file in glob.glob("*"):
    print(file)

In [ ]:
edges_network  = gpd.read_file('edges_d.shp')

In [ ]:
path_vertices = ""

In [ ]:
os.chdir(path_vertices)
for file in glob.glob("*"):
    print(file)

In [ ]:
vertices_network  = gpd.read_file('vertices_c.shp')

In [ ]:
path_neighb = ""

In [ ]:
os.chdir(path_neighb)
for file in glob.glob("*"):
    print(file)

In [ ]:
#import a file containing the neighbourhood polygon geometries and names
Neighbourhoo_geo = gpd.read_file('Neighbourhoods.shp')

In [ ]:
Neighbourhoo_geo.head()

In [ ]:
#change crs to required crs
vertices_network = vertices_network.to_crs(4326)
edges_network = edges_network.set_crs(3857)
edges_network = edges_network.to_crs(4326)
Neighbourhoo_geo = Neighbourhoo_geo.set_crs(4326)
Neighbourhoo_geo = Neighbourhoo_geo.to_crs(4326)

In [ ]:
#create x and y coordinates from geometry of vertices
vertices_network['x'] = vertices_network['geometry'].x
vertices_network['y'] = vertices_network['geometry'].y

## Add the neighbourhood names as an attribute to the vertices

In [ ]:
Neighb_names = Neighbourhoo_geo[['SP_NAME', 'geometry']]

In [ ]:
# Perform spatial join
# This assigns the neighborhood attributes to points
vertices_network2 = gpd.sjoin(vertices_network, Neighb_names, how='left', predicate='within')

In [ ]:
vertices_network2 #the vertices that do not fall in any neighbourhoods and do not matter

In [ ]:
vertices_network2.columns

In [ ]:
vertices_network2 = vertices_network2[['SP_NAME','land use', 'vertex_typ', 'x', 'y', 'id', 'geometry']]

## 3. Create a graph in network x from these dataframes

We create a graph of the edges and vertices in NetworkX so that we can calculate Reach Centrality. We create a directed graph because some streets and transport edges have specific directions of travel and varying weights depending on the direction of travel.

**1. Creating a directed graph**

In [ ]:
#create a directed graph 
G4 = nx.DiGraph()

In [ ]:
edges_network

In [ ]:
##adding the edges 

for index, row in edges_network.iterrows():
    source = row['source']
    target = row['target']
    time_cost = row['time_cost']
    length = row['Length']
    edge_type = row['edge_type']
    ID = row['id']
    vertex_type = row['vertex_typ']
    geometry = row['geometry']
    G4.add_edge(source, target, time_cost=time_cost, length=length, edge_type= edge_type, 
                 ID = ID, vertex_type = vertex_type, geometry=geometry)

In [ ]:
## adding the vertices

for index, row in vertices_network2.iterrows():
    # Extract the attributes from the dataframe row
    vertex_id = row['id']
    landuse = row['land use']
    neighbourhood = row['SP_NAME']
    vertex_type = row['vertex_typ']
    pos =(row['x'], row['y'])
    
    # Add the vertex to the graph with attributes
    G4.add_node(vertex_id, landuse=landuse, vertex_type = vertex_type, neighbourhood = neighbourhood,pos=pos)

In [ ]:
#check their length
len(G4.nodes)

**2. Deleting vertices with no geometry, this might have occurred through edges that were included but the vertices were not in neighbourhods which are in the metropolitan area**

In [ ]:
# create for loop to delete vertices that have no geometry

vertices_to_delete = []
for vertex in G4.nodes:
    if 'pos' not in G4.nodes[vertex]:
        vertices_to_delete.append(vertex)

In [ ]:
# Delete vertices that have no geometry
G4.remove_nodes_from(vertices_to_delete)

len(G4.nodes)

## 4. Preparing the Graph for Neighbourhood Reach Centrality

In this section:

- 1. Define the target vertices
- 2. Define the source attribute
- 3. Create the target vertices
- 4. Group the vertices by neighbourhood


#### 1. Define the target vertices

In [ ]:
#Creating a list of the target attributes, which is the land use types, this would be all land use types you 
#are interested in calculating access for
TARGET_ATTRIBUTES = ['General Business 1',
 'General Business 2',
 'General Business 3',
 'General Business 4',
 'General Business 5',
 'General Business 6',
 'General Business 7',
'Local Business 1 : Intermediate Business',
 'Local Business 2 : Local Business',
'Mixed Use 1',
 'Mixed Use 2',
 'Mixed Use 3',
'General Industrial 1',
 'General Industrial 2']

#### 2. Define the source attribute

In [ ]:
#The major source attribute of a vertex is that it must be a street
source_attribute = 'street'

**3. Create the Target vertices**

In [ ]:
#Create Target vertices

target_attribute = TARGET_ATTRIBUTES

target_vertices = []

for x in target_attribute:
    targets = [vertex for vertex, attr in G4.nodes(data=True) if attr.get('landuse') == x]
    for y in targets:
        target_vertices.append(y)

In [ ]:
target_vertices

#### 4. Group the vertices by neighbourhood

In [ ]:
#Create a list of the neighbourhood names which will be used to group the vertices
Neighbourhoods = list(vertices_network2['SP_NAME'].unique())

In [ ]:
len(Neighbourhoods)

In [ ]:
Neighbourhoods = [x for x in Neighbourhoods if str(x) != 'nan'] #removing nan as not relevant

In [ ]:
len(Neighbourhoods)

In [ ]:
#create lists of the neighbourhood vertices which will be used for the multi-source algorithm, as 
#the vertices will be grouped by neighbourhood

source_vertices_lists = []
for z in Neighbourhoods:
    source_vertices = [vertex for vertex, attr in G4.nodes(data=True) if attr.get('vertex_type') == source_attribute and attr.get('neighbourhood') == z]
    source_vertices_lists.append(source_vertices)

In [ ]:
source_vertices_lists

In [ ]:
len(source_vertices_lists)

In [ ]:
# Using list comprehension to filter out any empty lists
filtered_list = [sublist for sublist in source_vertices_lists if sublist]

In [ ]:
len(filtered_list)

In [ ]:
len(Neighbourhoods)

In [ ]:
Neighbourhoods

## 5. Calculating the Neighbourhood Reach Centrality

Calculating Reach:

- Reach Centrality at 15 min
- Reach Centrality at 30 min
- Reach Centrality at 45 min
- Reach Centrality at 60 min

In [ ]:
#creating the empty lists to populate with Reach values
Reach_15 = []
Reach_30 = []
Reach_45 = []
Reach_60 = []

In [ ]:
Reach_15

In [ ]:
help(sj.calculate_reach_centrality)

In [ ]:
#Reach 15
sj.calculate_reach_centrality(G4, filtered_list, target_vertices, 15, Reach_15, "time_cost", 15)

In [ ]:
#Reach 30
sj.calculate_reach_centrality(G4, source_vertices_lists, target_vertices, 30, Reach_30, "time_cost", 15)

In [ ]:
#Reach 45
sj.calculate_reach_centrality(G4, source_vertices_lists, target_vertices, 45, Reach_45, "time_cost", 15)

In [ ]:
#Reach 60
sj.calculate_reach_centrality(G4, source_vertices_lists, target_vertices, 60, Reach_60, "time_cost", 15)

### Create a dataframe

In [ ]:
Reach = pd.DataFrame({
    'SP_NAME': Neighbourhoods,
    'reach_15': Reach_15,
    'reach_30': Reach_30,
    'reach_45': Reach_45,
    'reach_60': Reach_60
})

In [ ]:
Reach

### Combining

I add the Reach values to the neighbourhood geometry file

In [ ]:
# Merging the DataFrames on the 'neighbourhood' column
df_combined = pd.merge(Reach, Neighbourhoo_geo, on='SP_NAME')

In [ ]:
df_combined.head()

In [ ]:
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df_combined, geometry='geometry')

## Export 

In [ ]:
path_to_save = " "

In [ ]:
os.chdir(path_to_save)
for file in glob.glob("*"):
    print(file)

In [ ]:
gdf.to_file('Neighbourhood_E.shp')